# Reports Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob
import csv

import json
import unittest

from common import Printable, testExit, projdir

from entrant import Entrant
from filter import Filter
from speedrun import SpeedRun

from constants import *

## Constants

Constants unique to this module

In [2]:
T_SORT_ASC = 'ASC'
T_SORT_DSC = 'DSC'

## Report Class

Class to manage a single report

In [3]:
class Report(Printable):
    def __init__(self, config, entrantsFilter, verbosity=1):
        '''Initialise report object'''    

        super().__init__(verbosity=verbosity)
        
        self.config = config.copy()
        self.entrantsFilter = entrantsFilter

        self.entrantFields = set()
        self.runFields = set()


    def patchConfig(self, config):
        '''Patch config with overrides - e.g. period specific or global / event'''
        
        for key in config:
            # Do not copy Periods or Reports from the global config
            if key not in ['Periods', 'Reports'] and key not in self.config:
                self.config[key] = config[key]


    def inspectFields(self):
        '''Determine which fields relate to the entrant or an individual run'''
        
        # Use a random entrant to determine the available fields
        entrantId = list(self.entrantsFilter.entrants.keys())[0]
        entrantFields = self.entrantsFilter.entrants[entrantId].entrantDict.keys()

        for field in self.fields:
            if field in entrantFields:
                self.entrantFields.add(field)
            else:
                self.runFields.add(field)
        

    def parseSort(self):
        '''Determine what will be done by the sort'''
        
        self.sort = []

        if 'Sort' in self.config:
            sortClauses = self.config['Sort'].split(',')
            for sortClause in sortClauses:
                # Convert sort clause to a tuple for later use
                clauseParts = tuple(sortClause.split('='))
                
                # Validate the sort order - slightly lazy approach
                if clauseParts[1] not in [T_SORT_ASC, T_SORT_DSC]:
                    raise ValueError('Invalid sort order for report "{}" - {}'.format(self.title, clauseParts[1]))

                # Sort clauses might relate to fields that are hidden in this mode of reporting
                if clauseParts[0] not in self.hideFields:
                    if clauseParts[0] not in self.fields:
                        raise ValueError('Invalid sort field for report "{}" - {}'.format(self.title, clauseParts[0]))

                    self.sort.append(clauseParts)

        else:
            # Default sort order is based on fastest speed
            if 'Speed (kts)' in self.fields:
                self.sort.append(('Speed (kts)', T_SORT_DSC))

            # Run time is used as a tiebreaker (only in sort order, not rankings)
            if 'Time' in self.fields:
                self.sort.append(('Time', T_SORT_ASC))
            elif 'Start Time' in self.fields:
                self.sort.append(('Start Time', T_SORT_ASC))
            elif 'Finish Time' in self.fields:
                self.sort.append(('Finish Time', T_SORT_ASC))

        # Limit to two fields for the benefit of sortResults()
        if len(self.sort) > 2:
            raise ValueError('Too many sort fields for report "{}" - {}'.format(self.title, self.config['Sort']))

        # Create a sort specification which is numerical (field index) and boolean (reverse)
        self.sortSpec = []
        for clauseParts in self.sort:
            self.sortSpec.append((self.fields.index(clauseParts[0]), clauseParts[1] == T_SORT_DSC))


    def parseText(self):
        '''Process the titles'''

        if 'Title' in self.config:
            self.title = self.config['Title']
        else:
            raise ValueError('Missing title for report - {}'.format(self.config))

        if 'Period' in self.config:
            self.period = self.config['Period']
        else:
            self.period = 'Session'
            
        if self.period not in ['Course', 'Session', 'Event', 'Round']:
            raise ValueError('Invalid period for report "{}"- {}'.format(self.title, self.period))
        
        if 'Subtitle' in self.config:
            self.subtitle = self.config['Subtitle']
        else:
            self.subtitle = None

        if 'Suptitle' in self.config:
            self.suptitle = self.config['Suptitle']
        else:
            self.suptitle = None

        if 'Series' in self.config:
            self.series = self.config['Series']
        else:
            self.series = None
            
        # TODO - validate series?


    def parseLists(self):
        '''Process the lists'''

        if 'Fields' in self.config:
            self.fields = self.config['Fields'].split(',')
        else:
            raise ValueError('Missing field list for report "{}"'.format(self.title))

        if 'HideFields' in self.config:
            self.hideFields = set()
            for hideField in self.config['HideFields'].split(','):
                if hideField in self.fields:
                    self.hideFields.add(hideField)
                    self.fields.remove(hideField)
                else:
                    raise ValueError('Invalid field to be hidden for report "{}" - "{}"'.format(self.title, hideField))
        else:
            self.hideFields = set()

        self.inspectFields()

        if 'Filter' in self.config:
            self.filter = self.config['Filter'].split(',')
            self.entrants = self.entrantsFilter.getEntrants(self.config['Filter'])
        else:
            self.filter = []
            self.entrants = self.entrantsFilter.entrants

        # No need to validate filter spec as it is done withing the Filter class

        self.parseSort()


    def parseInts(self):
        '''Process the ints'''

        if 'Limit' in self.config:
            self.limit = int(self.config['Limit'])
        else:
            self.limit = 99999

        if 'Prizes' in self.config:
            self.prizes = int(self.config['Prizes'])
        else:
            self.prizes = 0

        if 'ShowSpeeds' in self.config:
            self.showSpeeds = int(self.config['ShowSpeeds'])
        else:
            self.showSpeeds = 1


    def parseBools(self):
        '''Process the booleans'''

        def getBool(key):
            '''Convert Y / N to true / false'''

            value = self.config[key]

            if value not in [True, False]:
                if value[:1].lower() == 'n':
                    result = False
                elif value[:1].lower() == 'y':
                    result = True
            else:
                result = value

            if value not in [True, False]:
                raise ValueError('Bad value for "{}"'.format(entrant.getValue('ID')))

            return result

        if 'Profile' in self.config:
            self.profile = getBool('Profile')
        else:
            self.profile = False

        if 'Rank' in self.config:
            self.rank = getBool('Rank')
        else:
            self.rank = True

        if 'Trophy' in self.config:
            self.trophy = getBool('Trophy')
        else:
            self.trophy = False

        if 'Warning' in self.config:
            self.warning = getBool('Warning')
        else:
            self.warning = False


    def parseConfig(self):
        '''Process the configuration'''

        self.parseText()
        self.parseLists()
        self.parseInts()
        self.parseBools()
        
        del(self.config)


    def sortResults(self):
        '''Sort results according to specification - limited to two fields'''
        
        if len(self.sortSpec) == 1:
            index = self.sortSpec[0][0]
            if self.sortSpec[0][1] == True:
                self.results = sorted(self.results, key = lambda x: -1 * x[index])
            else:
                self.results = sorted(self.results, key = lambda x: x[index])

        elif len(self.sortSpec) == 2:
            index1 = self.sortSpec[0][0]
            index2 = self.sortSpec[1][0]

            if self.sortSpec[0][1] == True and self.sortSpec[1][1] == True:
                self.results = sorted(self.results, key = lambda x: (-1 * x[index1], -1 * x[index2]))
            elif self.sortSpec[0][1] == True and self.sortSpec[1][1] == False:
                self.results = sorted(self.results, key = lambda x: (-1 * x[index1], x[index2]))
            elif self.sortSpec[0][1] == False and self.sortSpec[1][1] == True:
                self.results = sorted(self.results, key = lambda x: (x[index1], -1 * x[index2]))
            else:
                self.results = sorted(self.results, key = lambda x: (x[index1], x[index2]))


    def printResults(self):
        '''Basic print of results for debugging purposes'''

        if len(self.results) > 0:
            print(self.title)
            print(self.fields)
            for result in self.results:
                print(result)
            print()


    def prepareResults(self, results):
        '''Process results by applying filter and sorting'''
        
        self.results = []

        for entrantId in results:
            speedRun = results[entrantId][0]
            if entrantId in self.entrants:
                entrant = self.entrants[entrantId]

                result = []
                for field in self.fields:
                    if field in self.entrantFields:
                        result.append(entrant.getValue(field))
                    elif field in speedRun.data:
                        result.append(speedRun.getValue(field))
                    else:
                        result.append('')

                self.results.append(result)
                
        self.sortResults()

## Reports Class

Class to manage a collection of reports

In [4]:
class Reports(Printable):
    def __init__(self, eventPath, entrants, verbosity=1):
        '''Initialise reports object'''    

        super().__init__(verbosity=verbosity)
        
        self.eventPath = eventPath
        self.entrants = entrants
        
        self.periods = {}
        
        self.entrantsFilter = Filter(entrants)


    def indexReports(self, filename, reports):
        '''Index the reports by period'''
        
        # Start by creating a single list for each period
        periods = {}
        for report in reports:
            period = report.period
            if period not in periods:
                periods[period] = []
            periods[period].append(report)

        # Finish off by adding the simple lists to the main collections
        basename = os.path.splitext(os.path.basename(filename))[0]
        for period in periods:
            if period not in self.periods:
                self.periods[period] = {}
            self.periods[period][basename] = periods[period]

        
    def loadReports(self):
        '''Load report configurations from JSON files'''
        
        filenames = glob.glob(os.path.join(os.path.join(self.eventPath, REPORTS_DIR), '*.json'))
        
        for filename in filenames:
            with open(filename, 'r', encoding='utf-8') as f:
                jsonTxt = f.read()
                try:
                    reportsConfig = json.loads(jsonTxt)
                except:
                    self.logError('Could not parse {}'.format(filename))
                    raise

                reports = []

                for reportConfig in reportsConfig['Reports']:
                    if 'Periods' in reportsConfig:
                        for periodConfig in reportsConfig['Periods']:
                            if 'Period' not in reportConfig or reportConfig['Period'] == periodConfig['Period']:
                                report = Report(reportConfig, self.entrantsFilter)
                                report.patchConfig(periodConfig)
                                report.patchConfig(reportsConfig)
                                report.parseConfig()                           
                                reports.append(report)
                    else:
                        report = Report(reportConfig, self.entrantsFilter)
                        report.patchConfig(reportsConfig)
                        report.parseConfig()
                        reports.append(report)

                self.indexReports(filename, reports)

## Unit Tests

A handful of basic filter tests, utilising a dummy "event" class

In [5]:
class DummyEvent(Printable):
    def __init__(self, path, verbosity=1):
        
        super().__init__(verbosity=verbosity)

        self.path = path
        
        self.entrants = {}


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r', encoding='utf-8') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(headers, values, verbosity=self.verbosity)
                if entrant.getValue('ID') not in self.entrants:
                    self.entrants[entrant.getValue('ID')] = entrant
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        self.logInfo('{} entrants loaded'.format(len(self.entrants)))


    def loadReports(self):
        '''Read reports from JSON files'''

        self.reports = Reports(self.path, self.entrants)

        self.reports.loadReports()

In [6]:
class TestReports1998(unittest.TestCase):
    '''Class to test reports using 2019 data'''

    def testPeriods(self):
        '''Test the periods have been indexed'''

        periods = event.reports.periods.keys()
        self.assertEqual('Session' in periods, True)
        self.assertEqual('Event' in periods, True)


    def testSessionReports(self):
        '''Test the session reports have been indexed'''

        reports = event.reports.periods['Session'].keys()
        self.assertEqual('fastest' in reports, True)
        self.assertEqual('runs' in reports, True)
        self.assertEqual('verification' in reports, True)
        self.assertEqual('craft' in reports, True)


    def testEventReports(self):
        '''Test the event reports have been indexed'''

        reports = event.reports.periods['Event'].keys()
        self.assertEqual('fastest' in reports, True)
        self.assertEqual('entrants' in reports, True)
        self.assertEqual('craft' in reports, True)


    def testSessionFastest(self):
        '''Test the session fastest'''

        fastest = event.reports.periods['Session']['fastest']
        self.assertEqual(len(fastest), 4)

        for report in fastest:
            self.assertEqual(report.period, 'Session')
            self.assertEqual('Name' in report.fields, True)
            self.assertEqual('Course' in report.fields, True)


    def testEventFastest(self):
        '''Test the event fastest'''

        fastest = event.reports.periods['Event']['fastest']
        self.assertEqual(len(fastest), 4)

        for report in fastest:
            self.assertEqual(report.period, 'Event')
            self.assertEqual('Name' in report.fields, True)
            self.assertEqual('Course' in report.fields, False)


    def testSuptitles(self):
        '''Test the suptitles'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            self.assertNotEqual(report.suptitle, None)


    def testSubtitles(self):
        '''Test the subtitles'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title.endswith('Overall'):
                self.assertEqual(report.subtitle, None)
            else:
                self.assertNotEqual(report.subtitle, None)


    def testFilters(self):
        '''Test the filters'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title.endswith('Overall'):
                self.assertEqual(report.filter, [])
            else:
                self.assertNotEqual(report.filter, [])


    def testLimits(self):
        '''Test the limits'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title.endswith('Overall'):
                self.assertEqual(report.limit, 10)
            else:
                self.assertEqual(report.limit, 3)


    def testWarnings(self):
        '''Test the warnings'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if 'Unclassified' in report.title:
                self.assertEqual(report.warning, True)
            else:
                self.assertEqual(report.warning, False)


    def testResults(self):
        '''Test result processing using just a couple of results'''

        headers = ['Date', 'Time', 'Sail Number', 'Speed (kts)', 'Wind Speed', 'Gust Speed', 'Ratio']

        # Torix Bennett
        torix = event.entrants[26]
        values = ['1998-10-06', '15:28:59', '101', '19.315', '16.00', '0.00', '1.207']
        torixRun = SpeedRun(None, torix, headers, values)

        # Nick Povey
        nick = event.entrants[27]
        values = ['1998-10-06', '15:33:10', '111', '21.922', '12.00', '0.00', '1.827']
        nickRun = SpeedRun(None, nick, headers, values)

        # Jean-Bernard Cunin
        jean = event.entrants[34]
        values = ['1998-10-06', '16:06:48', '584', '24.273', '16.00', '0.00', '1.517']
        jeanRun = SpeedRun(None, nick, headers, values)

        # Nick is fastest so results should be sorted accordingly
        results = {
            torix.getValue('ID'): [torixRun],
            nick.getValue('ID'): [nickRun, nickRun],
            jean.getValue('ID'): [jeanRun, jeanRun, jeanRun]
        }

        for periodName in event.reports.periods:
            period = event.reports.periods[periodName]
            for reportsName in period:
                reports = period[reportsName]
                for report in reports:
                    report.prepareResults(results)
                    # TODO - automate instead of using report.printResults()

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [7]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

    # Only load the 1998 event once for the unit tests
    eventPath = os.path.join(projdir, EVENTS_DIR, '1998')
    event = DummyEvent(eventPath)
    event.loadEntrants()
    event.loadReports()

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

...........
----------------------------------------------------------------------
Ran 11 tests in 0.023s

OK


## All Done!